# Time series forecasting EDA

Note: validation data available on kaggle competition page so need to submit to evaluate our model. 

## Objectives

Participants of M5 are asked to provide 28 days ahead point forecasts (PFs) for all the
series of the competition, as well as the corresponding median and 50%, 67%, 95%, and 99% prediction intervals (PIs).

More info at [M5 Competitors Guide](https://mofc.unic.ac.cy/m5-competition/)

## Download dataset from kaggle with kagle API

see [download_from_kaggle](../docs/download_from_kaggle.md)



## analyse

In [1]:
import lux
import polars as pl

In [2]:
# import dtale

# calendar_df = pd.read_csv('../data/raw/calendar.csv')
# d = dtale.show(calendar_df)
# d.open_browser()

In [3]:
# ctx = pl.SQLContext(
#     calendar=pl.scan_csv("../data/raw/calendar.csv"),
#     train=pl.scan_csv("../data/raw/sales_train_evaluation.csv"),
#     prices=pl.scan_csv("../data/raw/sell_prices.csv"),
#     eager_execution=True,
# )

In [4]:
-

In [8]:
import lux
calendar_pd = calendar.to_pandas()
calendar_pd

/home/ubuntu/miniforge3/envs/forecasting/lib/python3.9/site-packages/lux/executor/PandasExecutor.py:567: UserWarning:Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
/home/ubuntu/miniforge3/envs/forecasting/lib/python3.9/site-packages/lux/executor/PandasExecutor.py:548: UserWarning:Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
/home/ubuntu/miniforge3/envs/forecasting/lib/python3.9/site-packages/lux/executor/PandasExecutor.py:567: UserWarning:Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
/home/ubuntu/miniforge3/envs/forecasting/lib/python3.9/site-packages/lux/executor/PandasExecutor.py:548: UserWarning:Could not infer for

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,None,None,None,None,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,None,None,None,None,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,None,None,None,None,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,None,None,None,None,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,None,None,None,None,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,None,None,None,None,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,None,None,None,None,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,None,None,None,None,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,None,None,None,None,0,0,0


: 

In [6]:
print(f"{calendar.shape = }")
print(f"{train.shape = }")
print(f"{prices.shape = }")

calendar.shape = (1969, 14)
train.shape = (30490, 1947)
prices.shape = (6841121, 4)


In [7]:
# print(calendar.columns)
# print(train.columns[:10])
print(prices.columns)

['store_id', 'item_id', 'wm_yr_wk', 'sell_price']


calendar.columns =
['date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 'd', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI']

train.columns[:10] = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1', 'd_2', 'd_3', 'd_4']
d_1 .. d_1941 for day_1 to day_1941

prices.columns = 
['store_id', 'item_id', 'wm_yr_wk', 'sell_price']

In [8]:
query = """
SELECT
    prices.store_id, prices.item_id, prices.wm_yr_wk, sell_price, 
    id, train.item_id, dept_id, cat_id, train.store_id, state_id,
    date, calendar.wm_yr_wk, weekday, wday, month, year, d,
    event_name_1, event_type_1, event_name_2, event_type_2,
    snap_CA, snap_TX, snap_WI
FROM prices
LEFT JOIN train ON prices.store_id = train.store_id 
        AND prices.item_id = train.item_id
LEFT JOIN calendar on prices.wm_yr_wk = calendar.wm_yr_wk
"""

# print(ctx.execute(query)) # results in error below

print(ctx.execute(query)) 

InvalidOperationError: SQL join constraint On(BinaryOp { left: BinaryOp { left: CompoundIdentifier([Ident { value: "prices", quote_style: None }, Ident { value: "store_id", quote_style: None }]), op: Eq, right: CompoundIdentifier([Ident { value: "train", quote_style: None }, Ident { value: "store_id", quote_style: None }]) }, op: And, right: BinaryOp { left: CompoundIdentifier([Ident { value: "prices", quote_style: None }, Ident { value: "item_id", quote_style: None }]), op: Eq, right: CompoundIdentifier([Ident { value: "train", quote_style: None }, Ident { value: "item_id", quote_style: None }]) } }) is not yet supported

In [ ]:
prices_calendar = prices.join(calendar, on="wm_yr_wk", how="left", )
print(prices_calendar.head())
# results in error below

: 

: 

: 

Cannot execute code, session has been disposed. Please try restarting the Kernel.
The Kernel crashed while executing code in the the current cell or a previous cell. Please review the code in the cell(s) to identify a possible cause of the failure. Click here for more info. View Jupyter log for further details.
Cannot execute code, session has been disposed. Please try restarting the Kernel.

full df has 1965 columns!